In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [2]:
# for repreoducibility
torch.manual_seed(1)

In [3]:
x_train = torch.FloatTensor([[1 ,2, 1],
                             [1 ,3, 2],
                             [1 ,3, 4],
                             [1 ,5, 5],
                             [1 ,7, 5],
                             [1 ,2, 5],
                             [1 ,6, 6],
                             [1 ,7, 7],
                            ])
y_train = torch.LongTensor([2, 2, 2, 1, 1, 1, 0, 0])
x_test = torch.FloatTensor([[2, 1, 1], [3, 1, 2], [3, 3, 4]])
y_test = torch.LongTensor([2, 2, 2])

In [4]:
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(3, 3)
        
    def forward(self, x):
        return self.linear(x)

In [5]:
model = SoftmaxClassifierModel()
optimizer = optim.SGD(model.parameters(), lr=0.1)

In [9]:
def train(model, optimizer, x_train, y_train):
    n_epochs = 20
    for epoch in range(n_epochs):

        prediction = model(x_train)
        cost = F.cross_entropy(prediction, y_train)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, n_epochs, cost.item()))
        
def test(model, optimizer, x_test, y_test):
    prediction = model(x_test)
    cost = F.cross_entropy(prediction, y_test)
    predicted_classes = prediction.max(1)[1]
    accuracy = (predicted_classes == y_test).float().mean()
    
    print('Accuracy: {}% Cost: {:.6f}'.format(
        accuracy.item(), cost.item()))

In [7]:
train(model, optimizer, x_train, y_train)

Epoch    0/20 Cost: 2.203667
Epoch    1/20 Cost: 1.199645
Epoch    2/20 Cost: 1.142985
Epoch    3/20 Cost: 1.117769
Epoch    4/20 Cost: 1.100901
Epoch    5/20 Cost: 1.089523
Epoch    6/20 Cost: 1.079872
Epoch    7/20 Cost: 1.071320
Epoch    8/20 Cost: 1.063325
Epoch    9/20 Cost: 1.055720
Epoch   10/20 Cost: 1.048378
Epoch   11/20 Cost: 1.041245
Epoch   12/20 Cost: 1.034285
Epoch   13/20 Cost: 1.027478
Epoch   14/20 Cost: 1.020813
Epoch   15/20 Cost: 1.014279
Epoch   16/20 Cost: 1.007872
Epoch   17/20 Cost: 1.001586
Epoch   18/20 Cost: 0.995419
Epoch   19/20 Cost: 0.989365


In [10]:
test(model, optimizer, x_test, y_test) # overfitting

Accuracy: 0.0% Cost: 1.425844


In [21]:
def train_with_test(model, optimizer, x_train, y_train, x_test, y_test):
    n_epochs = 20
    prev_acc = 0.0
    for epoch in range(n_epochs):

        prediction = model(x_train)
        cost = F.cross_entropy(prediction, y_train)

        optimizer.zero_grad()
        cost.backward()
        optimizer.step()
        
        prediction = model(x_test)
        test_cost = F.cross_entropy(prediction, y_test)
        predicted_classes = prediction.max(1)[1]
        accuracy = (predicted_classes == y_test).float().mean().item()

        print('Train Epoch {:4d}/{} Cost: {:.6f} || Test Accuracy: {:.1f}% Cost: {:.6f}'.format(
            epoch, n_epochs, cost.item(), accuracy * 100, test_cost.item()))
        
        if prev_acc <= accuracy:
            prev_acc = accuracy
        else:
            break

In [22]:
model = SoftmaxClassifierModel()
optimizer = optim.SGD(model.parameters(), lr=0.1)
train_with_test(model, optimizer, x_train, y_train, x_test, y_test)

Train Epoch    0/20 Cost: 1.341574 || Test Accuracy: 0.0% Cost: 1.137680
Train Epoch    1/20 Cost: 1.198802 || Test Accuracy: 0.0% Cost: 1.011065
Train Epoch    2/20 Cost: 1.150877 || Test Accuracy: 33.3% Cost: 0.891487
Train Epoch    3/20 Cost: 1.131977 || Test Accuracy: 33.3% Cost: 0.886537
Train Epoch    4/20 Cost: 1.116242 || Test Accuracy: 33.3% Cost: 0.829073
Train Epoch    5/20 Cost: 1.102514 || Test Accuracy: 33.3% Cost: 0.804475
Train Epoch    6/20 Cost: 1.089676 || Test Accuracy: 66.7% Cost: 0.760810
Train Epoch    7/20 Cost: 1.077479 || Test Accuracy: 66.7% Cost: 0.732455
Train Epoch    8/20 Cost: 1.065775 || Test Accuracy: 66.7% Cost: 0.696384
Train Epoch    9/20 Cost: 1.054511 || Test Accuracy: 66.7% Cost: 0.668205
Train Epoch   10/20 Cost: 1.043655 || Test Accuracy: 66.7% Cost: 0.637083
Train Epoch   11/20 Cost: 1.033187 || Test Accuracy: 100.0% Cost: 0.610492
Train Epoch   12/20 Cost: 1.023091 || Test Accuracy: 100.0% Cost: 0.583053
Train Epoch   13/20 Cost: 1.013356 || 

### data processing
- standardization

$x^{'}_{j}=\frac{x_{j}-\mu_{j}}{\sigma_{j}}$ $\sim N(0,1)$

**데이터의 특성**에 따라 정규화 등의 전처리를 해 주면 학습이 더욱 수월해진다.
특히, y 가 2 차원 이상인 경우에는 정규화가 필수!